In [1]:
import pandas as pd
import csv
import os

In [2]:
datetime_format = "%d/%m/%Y %H:%M"

def csv_extract(filepath):
    with open(filepath, 'r') as file:
        reader = csv.reader(file)

        for i, row in enumerate(list(reader)):
            if row == []:
                cutOff = i - 1
                break
    
    with open(filepath, 'r') as file:
        csv_list = list(csv.DictReader(file))[:cutOff]

        df = pd.DataFrame(csv_list)
        df = df.rename(columns={
            "\ufeffAccount Number 編賬號碼": "AccID", 
            "Total Consumption (Unit) 總用電量 (度數)": "UnitConsumption", 
            "Start date/time 開始日期/時間": "StartTime", 
            "End date/time 結束日期/時間": "EndTime"
            })

        return df

def csv_parse_time(df) -> pd.DataFrame:
    df["StartTime"] = pd.to_datetime(df["StartTime"], format=datetime_format)
    df["EndTime"] = pd.to_datetime(df["EndTime"], format=datetime_format)
    return df

def parse(filepath) -> pd.DataFrame:
    df = csv_extract(filepath)
    df = csv_parse_time(df)
    return df


    

In [7]:
def filter_by_time_diff(df, t_minutes):
    """
    Filters a DataFrame to find rows where the time difference between
    'EndTime' and 'StartTime' is greater than or equal to t_minutes.

    Args:
        df (pd.DataFrame): DataFrame with 'StartTime' and 'EndTime' columns.
        t_minutes (int): The minimum time difference in minutes.

    Returns:
        pd.DataFrame: A DataFrame containing only the rows that meet the
                      time difference criteria.
    """
    # Ensure time columns are in datetime format
    df['StartTime'] = pd.to_datetime(df['StartTime'])
    df['EndTime'] = pd.to_datetime(df['EndTime'])

    # Calculate the time difference in minutes
    time_diff = (df['EndTime'] - df['StartTime']).dt.total_seconds() / 60

    # Filter the DataFrame
    return df[time_diff >= t_minutes]

In [4]:
filename = "../test_files/consumption_history.csv"

df = parse(filename)
print(df)

            AccID           StartTime             EndTime UnitConsumption
0     82865239130 2025-04-24 00:00:00 2025-04-24 01:00:00             0.4
1     82865239130 2025-04-24 01:00:00 2025-04-24 02:00:00             0.5
2     82865239130 2025-04-24 02:00:00 2025-04-24 03:00:00             0.4
3     82865239130 2025-04-24 03:00:00 2025-04-24 04:00:00             0.4
4     82865239130 2025-04-24 04:00:00 2025-04-24 05:00:00             0.4
...           ...                 ...                 ...             ...
2172  82865239130 2025-07-23 12:00:00 2025-07-23 13:00:00             0.1
2173  82865239130 2025-07-23 13:00:00 2025-07-23 14:00:00             0.1
2174  82865239130 2025-07-23 14:00:00 2025-07-23 15:00:00             0.1
2175  82865239130 2025-07-23 15:00:00 2025-07-23 16:00:00             0.1
2176  82865239130 2025-07-23 16:00:00 2025-07-23 17:00:00             0.1

[2177 rows x 4 columns]


In [6]:
check_time_difference_le_minute(df)

❌ Check failed: One or more rows have a time difference greater than 1 minute.

Rows with excessive time difference:
            AccID           StartTime             EndTime UnitConsumption
0     82865239130 2025-04-24 00:00:00 2025-04-24 01:00:00             0.4
1     82865239130 2025-04-24 01:00:00 2025-04-24 02:00:00             0.5
2     82865239130 2025-04-24 02:00:00 2025-04-24 03:00:00             0.4
3     82865239130 2025-04-24 03:00:00 2025-04-24 04:00:00             0.4
4     82865239130 2025-04-24 04:00:00 2025-04-24 05:00:00             0.4
...           ...                 ...                 ...             ...
2172  82865239130 2025-07-23 12:00:00 2025-07-23 13:00:00             0.1
2173  82865239130 2025-07-23 13:00:00 2025-07-23 14:00:00             0.1
2174  82865239130 2025-07-23 14:00:00 2025-07-23 15:00:00             0.1
2175  82865239130 2025-07-23 15:00:00 2025-07-23 16:00:00             0.1
2176  82865239130 2025-07-23 16:00:00 2025-07-23 17:00:00            

False